
This notebook is to convert the output of static analysis (e.g., static ad analysis) into the input of dynamic analysis (LLM-based UI exploration)



# Ad widgets

In [ ]:
First, we will read the json files from `data/json_output`, which is the output dir of static ad analysis. Keep o

In [ ]:
import warnings
import pandas as pd

In [ ]:
filenames=os.listdir('data/json_output/')
filenames=[ filename for filename in filenames if filename.endswith('.json')]

In [ ]:
import pandas as pd
from pandas import json_normalize

# Function to flatten the 'views' column and extract relevant information
def flatten_views(data):
    # Initialize an empty DataFrame to hold the flattened data
    flattened_data = pd.DataFrame()

    # Iterate through each row in the data
    for index, row in data.iterrows():
        # Normalize the 'views' column to flatten the nested structure
        flattened_views = json_normalize(row['views'])
        flattened_views['activity_name'] = row['name']  # Add activity name to each row
        # flatten_views['ad']
        # Append the flattened views to the aggregated DataFrame
        flattened_data = pd.concat([flattened_data, flattened_views], ignore_index=True)

    return flattened_data


In [ ]:
dfs=[]
statistics={'adview':0,'pop-up':0,'custom-made':0}
adlibs=["unityads", "facebook.ads", "gms.ads", "doubleclick", "google.ads", "flurry", "appbrain", "adcolony", "applovin", "inmobi.ads", "mbridge.msdk", "com.vungle", "applovin", "com.facebook", "bytedance.sdk", "com.bytedance.sdk", "com.applovin", "com.unity3d", "com.mbridge", "com.adjust", "easybrain", "mopub", "hyprmx", "verizon", "yandex", "smaato", "ironsource", "unity3d", "fyber", "net.pubnative.lite", "io.bidmachine", "inmobi", "bytedance", "io.presage", "com.baidu.mobads", "com.qq.e.ads", "com.kwad", "com.ss.android.downloadlib", "com.zero.flutter_gromore_ads", "com.cssq.ad", "com.youdao.sdk.extra.common"]
for filename in filenames:  
    outfile=f"data/csv/{filename.split('.')[0]}.csv"
    if os.path.exists(outfile):
        continue
    if filename.endswith('.json'):
        if os.path.getsize('data/json_output/'+filename) == 0:
            continue
        data=pd.read_json('data/json_output/'+filename)
        
        # Flatten the 'views' column of the loaded JSON data
        flattened_data = flatten_views(data)
        df=flattened_data[flattened_data.label!='none']
        if len(df.label)==0:
            # in this case, no widget is associated with view ID or event handler, meaning static ad analysis fail
            continue
        df[['type', 'ad-widget']] = df['label'].str.split('|',n=1, expand=True)
        df=df[['type', 'ad-widget','handlers','activity_name']]

        # Our static ad analysis may fail to exclude the widgets from 3rd-party libraries, such as ad libraries (we only care about the app code write by the app developer)
        # Accidentally analyzing an ad library can produce tons of ad widgets, which are false positives (Based on our inference rules in paper, almost every widgets inside ad liararies' code will be regared as ad widgets)
        # So we remove these accicentally included ad libraries' widgets.
        df=df[~df['activity_name'].apply(lambda x: any([adlib in x for adlib in adlibs]))]  

        # some view id is dynamically assigned, so the static ad analysis will output NO_ID.
        # NO_ID can hardly provide any information to help the LLM-based exploration
        # So we remove the widgets with `No_ID`
        df=df[~df['ad-widget'].apply(lambda x: 'NO_ID' in x)]
        
        if len(df)>0:
            df.to_csv(outfile,index=False)



# WTG